In [1]:
import numpy as np 
import pandas as pd
import pandas_profiling

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [6]:
train = pd.read_csv("../data/train.csv", delimiter='\t')
test = pd.read_csv("../data/test.csv", delimiter='\t')

In [7]:
train.shape

(30500, 347)

In [8]:
test.shape

(4166, 347)

In [183]:
prof_report = pandas_profiling.ProfileReport(train)

In [7]:
prof_report.to_file("pp_report.html")

In [ ]:
rejected_features = prof_report.get_rejected_variables()

In [9]:
rejected_features = ['117',
 '119',
 '122',
 '146',
 '148',
 '153',
 '158',
 '165',
 '179',
 '180',
 '206',
 '323',
 '324',
 '325',
 '326',
 '327',
 '333',
 '334',
 '335',
 '341',
 '9']


In [10]:
test.shape[0] /train.shape[0] 

0.13659016393442622

# Validation

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [12]:
import lightgbm as lgbm

In [11]:
# Validation

new_train, valid = train_test_split(train, test_size=0.14, random_state=42)

In [16]:
X = train.drop("0", axis=1)
y = train["0"]

In [14]:
X_train = train.drop(cv_idx)
X_train.drop('0', axis=1, inplace=True)

y_train = train.drop(cv_idx)['0']

NameError: name 'cv_idx' is not defined

In [17]:
# SKF
skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)

In [26]:
cv_iter = skf.split(X, y)

In [12]:
train = train.drop(rejected_features, axis=1)

In [13]:
cv_idx = valid.index

In [15]:
features = train.drop('0', axis=1).columns

In [19]:
lgbm_model = lgbm.LGBMClassifier(
    n_estimators=300, 
    colsample_bytree=0.6, 
    subsample=0.7,
    learning_rate=0.01, 
    subsample_for_bin=300000,
    num_leaves=21
)

In [27]:
cross_val_score(lgbm_model, X, y, cv=cv_iter, verbose=4, scoring="roc_auc")

[CV]  ................................................................
[CV] ........................ , score=0.732106815711846, total=   4.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


[CV] ....................... , score=0.7379053694843167, total=   4.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.7s remaining:    0.0s


[CV] ....................... , score=0.7260974909787206, total=   4.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   12.7s remaining:    0.0s


[CV] ....................... , score=0.7320430669056659, total=   4.1s
[CV]  ................................................................
[CV] ....................... , score=0.7425757340522129, total=   4.8s
[CV]  ................................................................
[CV] ....................... , score=0.7277568037679801, total=   4.4s
[CV]  ................................................................
[CV] ........................ , score=0.737351824974163, total=   5.6s
[CV]  ................................................................
[CV] ....................... , score=0.7376951131111675, total=   5.1s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   36.7s finished


array([0.73210682, 0.73790537, 0.72609749, 0.73204307, 0.74257573,
       0.7277568 , 0.73735182, 0.73769511])

In [17]:
lgbm_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.6,
        learning_rate=0.01, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=21, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.7,
        subsample_for_bin=300000, subsample_freq=1)

In [18]:
cv_pred = lgbm_model.predict_proba(train.loc[cv_idx, features])[:, 1]

In [19]:
print("CV roc_auc: {:.6f}".format(roc_auc_score(train.loc[cv_idx, '0'], cv_pred)))

CV roc_auc: 0.743810


In [20]:
train_pred = lgbm_model.predict_proba(train.drop(cv_idx, 0)[features])[:, 1]

In [21]:
print("Train roc_auc: {:.6f}".format(roc_auc_score(train.drop(cv_idx, 0)['0'], train_pred)))

Train roc_auc: 0.772196


In [448]:
params = "n_estimators=300, colsample_bytree=0.6, subsample=0.7, learning_rate=0.005, subsample_for_bin=300000, num_leaves=21"

In [449]:
model = []

In [425]:
cv_scores = []
train_scores = []
lb_scores = []

In [450]:
model.append(params)

In [436]:
cv_scores.append(roc_auc_score(train.loc[cv_idx, '0'], cv_pred))
train_scores.append(roc_auc_score(train.drop(cv_idx, 0)['0'], train_pred))

In [440]:
lb_scores.append(0.73535022)

In [452]:
results = pd.DataFrame()

In [453]:
results['cv_score'] = cv_scores

In [454]:
results['train_score'] = train_scores

In [455]:
results['public_lb'] = lb_scores

In [458]:
results['model'] = model

ValueError: Length of values does not match length of index

In [22]:
pred = (lgbm_model.predict_proba(test[features])[:, 1])

In [447]:
results

,cv_score,train_score
0,0.741477,0.763475
1,0.736375,0.744921


# XGBoost

max_depth=5, n_estimators=200, learning_rate=0.065

In [36]:
import xgboost as xgb

In [37]:
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.035,
    max_depth=5,
    random_state=42,

    #colsample_bytree = 0.7,
    #colsample_bylevel = 0.7,
    
    #max_leaves = 8,
    
    #tree_method = 'hist',
    #grow_policy = 'lossguide'
)

In [38]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.035, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [194]:
cv_pred = xgb_model.predict_proba(train.loc[cv_idx, features])[:, 1]

print("XGB CV roc_auc: {:.6f}".format(roc_auc_score(train.loc[cv_idx, '0'], cv_pred)))

train_pred =  xgb_model.predict_proba(train.drop(cv_idx, 0)[features])[:, 1]

print("XGB Train roc_auc: {:.6f}".format(roc_auc_score(train.drop(cv_idx, 0)['0'], train_pred)))

XGB CV roc_auc: 0.737446
XGB Train roc_auc: 0.865238


In [39]:
pred = (xgb_model.predict_proba(test[features])[:, 1])

Submission

In [32]:
submission = pd.DataFrame(test.iloc[:, 0])

In [33]:
submission.rename(index=str, columns={'Unnamed: 0': '_ID_'}, inplace=True)

In [34]:
submission['_VAL_'] = pred

In [169]:
submission

,_ID_,_VAL_
0,0,0.187420
1,1,0.427704
2,2,0.257056
3,3,0.299153
4,4,0.495362
5,5,0.252427
6,6,0.103507
7,7,0.157629
8,8,0.111361
9,9,0.316885


In [35]:
submission.to_csv("cat_best.csv", index=False)

In [ ]:
# lgbm: cv - 0.736375, pl - 0.73535022
# xgb: cv - 0.739591, train - 0.744921, lb - 0.76068500
# 0.741669 - 0.822175
"""
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.75,
       colsample_bytree=0.7, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
"""

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.01,
    n_jobs=4,
    max_depth = 3,
    
    colsample_bytree = 0.7,
    colsample_bylevel = 0.7
)
XGB CV roc_auc: 0.736033
XGB Train roc_auc: 0.743417
LB: 0.72961263

xgb_model = xgb.XGBClassifier(
    n_estimators=363,
    learning_rate=0.02,
    n_jobs=4,
    max_depth = 3,

    colsample_bytree = 0.7,
    colsample_bylevel = 0.7,
    
    max_leaves = 32,
    
    tree_method = 'hist',
    grow_policy = 'lossguide'
)

XGB CV roc_auc: 0.739078
XGB Train roc_auc: 0.755951
LB: 0.73779452

xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.02,
    n_jobs=4,
    max_depth = 3,

    colsample_bytree = 0.75,
    colsample_bylevel = 0.7,
    
    max_leaves = 32,
    
    tree_method = 'hist',
    grow_policy = 'lossguide'
)

XGB CV roc_auc: 0.740545
XGB Train roc_auc: 0.797018
LB: 0.74823467

xgb_model = xgb.XGBClassifier(
    n_estimators=350,
    learning_rate=0.03,
    n_jobs=4,
    max_depth=3,

    colsample_bytree = 0.75,
    colsample_bylevel = 0.7,
    
    max_leaves = 18,
    
    tree_method = 'hist',
    grow_policy = 'lossguide'
)

XGB CV roc_auc: 0.743138
XGB Train roc_auc: 0.768881
LB: 0.74738783

xgb_model = xgb.XGBClassifier(
    n_estimators=373,
    learning_rate=0.03,
    n_jobs=4,
    max_depth=5,
    random_state=42,

    colsample_bytree = 0.75,
    colsample_bylevel = 0.7,
    
    max_leaves = 8,
    
    tree_method = 'hist',
    grow_policy = 'lossguide'
)

XGB CV roc_auc: 0.745024
XGB Train roc_auc: 0.778765

# CatBoost

In [27]:
import catboost as ctb

In [41]:
ctb_model = ctb.CatBoostClassifier(
    loss_function = 'Logloss',
    eval_metric='AUC',

    iterations = 350,
    learning_rate = 0.023,
    random_seed = 42,
    use_best_model = False,
    
    od_wait = 50,
    od_type ='Iter',
    
    rsm = 0.7,
    l2_leaf_reg=2
)

In [42]:
ctb_data = ctb_model.fit(X_train, y_train)

0:	learn: 0.6040846	total: 85.5ms	remaining: 29.8s
1:	learn: 0.6196742	total: 214ms	remaining: 37.2s
2:	learn: 0.6385400	total: 340ms	remaining: 39.4s
3:	learn: 0.6571854	total: 469ms	remaining: 40.6s
4:	learn: 0.6648100	total: 604ms	remaining: 41.7s
5:	learn: 0.6646004	total: 709ms	remaining: 40.6s
6:	learn: 0.6686814	total: 826ms	remaining: 40.5s
7:	learn: 0.6706929	total: 965ms	remaining: 41.3s
8:	learn: 0.6712633	total: 1.1s	remaining: 41.6s
9:	learn: 0.6743800	total: 1.2s	remaining: 40.9s
10:	learn: 0.6764866	total: 1.32s	remaining: 40.6s
11:	learn: 0.6752751	total: 1.49s	remaining: 42.1s
12:	learn: 0.6796813	total: 1.75s	remaining: 45.3s
13:	learn: 0.6793699	total: 1.94s	remaining: 46.5s
14:	learn: 0.6802737	total: 2.1s	remaining: 47s
15:	learn: 0.6805010	total: 2.22s	remaining: 46.4s
16:	learn: 0.6836760	total: 2.35s	remaining: 46.1s
17:	learn: 0.6840082	total: 2.46s	remaining: 45.4s
18:	learn: 0.6834383	total: 2.57s	remaining: 44.7s
19:	learn: 0.6836489	total: 2.74s	remaining: 

161:	learn: 0.7372615	total: 22.5s	remaining: 26.1s
162:	learn: 0.7374194	total: 22.6s	remaining: 25.9s
163:	learn: 0.7376148	total: 22.8s	remaining: 25.8s
164:	learn: 0.7377812	total: 23s	remaining: 25.7s
165:	learn: 0.7380251	total: 23.2s	remaining: 25.7s
166:	learn: 0.7381936	total: 23.4s	remaining: 25.7s
167:	learn: 0.7384534	total: 23.6s	remaining: 25.5s
168:	learn: 0.7386444	total: 23.7s	remaining: 25.4s
169:	learn: 0.7388212	total: 23.8s	remaining: 25.2s
170:	learn: 0.7389059	total: 24s	remaining: 25.1s
171:	learn: 0.7390595	total: 24.1s	remaining: 25s
172:	learn: 0.7392021	total: 24.3s	remaining: 24.9s
173:	learn: 0.7393174	total: 24.5s	remaining: 24.7s
174:	learn: 0.7396203	total: 24.6s	remaining: 24.6s
175:	learn: 0.7398571	total: 24.7s	remaining: 24.4s
176:	learn: 0.7398768	total: 24.9s	remaining: 24.3s
177:	learn: 0.7400470	total: 25s	remaining: 24.2s
178:	learn: 0.7402729	total: 25.2s	remaining: 24s
179:	learn: 0.7404252	total: 25.3s	remaining: 23.9s
180:	learn: 0.7405140	

321:	learn: 0.7605418	total: 44.9s	remaining: 3.91s
322:	learn: 0.7606925	total: 45.1s	remaining: 3.77s
323:	learn: 0.7608197	total: 45.2s	remaining: 3.63s
324:	learn: 0.7609472	total: 45.3s	remaining: 3.48s
325:	learn: 0.7610702	total: 45.4s	remaining: 3.35s
326:	learn: 0.7611884	total: 45.6s	remaining: 3.2s
327:	learn: 0.7613336	total: 45.7s	remaining: 3.06s
328:	learn: 0.7614618	total: 45.8s	remaining: 2.92s
329:	learn: 0.7615879	total: 45.9s	remaining: 2.78s
330:	learn: 0.7616549	total: 46s	remaining: 2.64s
331:	learn: 0.7617031	total: 46.2s	remaining: 2.5s
332:	learn: 0.7618108	total: 46.3s	remaining: 2.36s
333:	learn: 0.7619276	total: 46.4s	remaining: 2.22s
334:	learn: 0.7619978	total: 46.6s	remaining: 2.08s
335:	learn: 0.7621523	total: 46.7s	remaining: 1.95s
336:	learn: 0.7623130	total: 46.8s	remaining: 1.81s
337:	learn: 0.7623974	total: 47s	remaining: 1.67s
338:	learn: 0.7624768	total: 47.1s	remaining: 1.53s
339:	learn: 0.7625243	total: 47.2s	remaining: 1.39s
340:	learn: 0.7626

In [30]:
cv_pred = ctb_model.predict_proba(train.loc[cv_idx, features])[:, 1]

print("Cat CV roc_auc: {:.6f}".format(roc_auc_score(train.loc[cv_idx, '0'], cv_pred)))

train_pred =  ctb_model.predict_proba(train.drop(cv_idx, 0)[features])[:, 1]

print("Cat Train roc_auc: {:.6f}".format(roc_auc_score(train.drop(cv_idx, 0)['0'], train_pred)))

Cat CV roc_auc: 0.741563
Cat Train roc_auc: 0.763584


In [43]:
pred = (ctb_model.predict_proba(test[features])[:, 1])

Cat CV roc_auc: 0.743388
Cat Train roc_auc: 0.796403
LB: 0.74828599

iterations = 300,
learning_rate = 0.05,
random_seed = 42,
use_best_model = False,

od_wait = 50,
od_type ='Iter'


Cat CV roc_auc: 0.742080
Cat Train roc_auc: 0.765468
LB; 0.73790573

iterations = 350,
    learning_rate = 0.023,
    random_seed = 42,
    use_best_model = False,
    
    od_wait = 50,
    od_type ='Iter',
    
    rsm = 0.7,
    l2_leaf_reg=2
   

In [177]:
lgbm_pred = pred

In [40]:
xgb_pred = pred

In [44]:
ctb_pred = pred

In [178]:
ensemble_cv = xgb_cv*0.4+lgbm_cv*0.3+ctb_cv*0.3
print("Simple weighted ensemble CV: {:.6f}".format(roc_auc_score(train.loc[cv_idx, '0'], ensemble_cv)))

Simple weighted ensemble CV: 0.743884


In [164]:
pred = xgb_pred*0.3+lgbm_pred*0.5+ctb_pred*0.3

In [49]:
len(xgb_pred)

4166

In [51]:
skf = StratifiedKFold(n_splits=7)

In [55]:


xgb_predictions = []
ctb_predictions = []

for train, val in skf.split(X_train, y_train):
    xgb_model.fit(X_train.iloc[train], y_train[train])
    ctb_model.fit(X_train.iloc[train], y_train[train])
    xgb_predictions.append([y_train[val], xgb_predictions.predict_proba(X_train.iloc[val])[:,1]])
    ctb_predictions.append([y_train[val], ctb_predictions.predict_proba(X_train.iloc[val])[:,1]])

/usr/local/lib/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


ValueError: y contains new labels: [nan nan nan ... nan nan nan]